
#   Übungsserie 1


Bevor wir die Daten Analysieren und an die phasen des CRISP-DM Prozessen durchgehen, Sollen wir erst mal zu unserer Fallstudie zurückgehen und die wichtigsten Punkte notieren.
####Aus der Fallstudie sind diese Punkte wichtig:
  - Mit Hilfe eines Computers einen Wetterbericht erstellen und Vorhersagen, was am tag zu machen.
  - Die Daten wurden über knap 2 Jahren morgens gesammelt.
  - Die Daten fassen Messung um zu:
          1.   Der releativen Luftfreutigkeit
          2.   Der Temperatur
          3.   Der gefallenen Niederschlagsmenge
          4.   Der Schneehöhe
  - Gesuchte Variable ist was zu unternehmen.

  Nach der Umfassung der wichtigsten Punkte könne wir durch CRISP-DM Prozesse gehen.
Durch CRISP-DM Modell definieren wir anhand unserer Fallstudie die Processe des CRISP-DM:



![titel](https://www.kdnuggets.com/wp-content/uploads/crisp-dm-4-problems-fig1.png)




## Business Understanding: 

-   das Ziel, wie es schon definiert ist, ist, dass Kurt durch vorhandene Daten 
eine kräftige Vorhersage machen kann, was er mit seinen Freunden unternehmen kann oder denen Vorschläge geben kann.
-   Es wird weiter durch CRISP-DM Prozesse vorgegangen
-   Es wird vogegeben das KNN Verfahren anzuvenden
-   Durch das Modell soll best möglich vorhersagen
-   Die Performance des Modells soll zwischen 85 % und 95 % liegen


## Data Understanding:

-   Die Daten sind for 2 Jahren gesammelt und enthalten 4 Messungen:



In [0]:
#### Importieren der Bibliotheken
import numpy as np
import pandas as pd   
from datetime import datetime as dt
import plotly.graph_objects as go    
import plotly.express as px 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif, chi2, mutual_info_classif
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
## Normalisierung
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import random
from sklearn.model_selection import cross_val_score
from flask import Flask, jsonify, request
import json
import base64
import pickle

In [2]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#### Lesen unseres Datesatzes

df = pd.read_csv("Daten_Projekt_Kommagetrennt.csv")
##  Dataframe 
df.head(len(df))

,id,Datum,relative Luftfeuchtigkeit,Temperatur,Niederschlag,Schnee,Freizeitangebot
0,1,01.01.2011,90.50,-0.6,0.0,58.0,Skifahren
1,2,08.01.2011,90.79,6.0,0.0,13.0,Wandern
2,3,15.01.2011,80.29,7.6,0.1,0.0,Wandern
3,4,22.01.2011,84.79,-2.9,0.0,2.0,Skifahren
4,5,29.01.2011,87.13,-7.5,0.0,2.0,Skifahren
...,...,...,...,...,...,...,...
100,101,01.12.2012,90.67,-3.9,0.0,8.0,Skifahren
101,102,08.12.2012,86.46,-9.0,0.0,6.0,Skifahren
102,103,15.12.2012,89.08,2.5,2.1,22.0,Skifahren
103,104,22.12.2012,95.04,1.4,4.6,2.0,Lesen



Hinweise:<br>
Für unser Model sind nicht alle Variable wichtig, deshalb werden Das Datum als index der Daten abgestellt und das ID gelöscht. 
<br>Der Grund Dafür ist, dass diese Variable unwiederholbar sind. d.h. die sind nicht signifikant für das model. Außerdem kann überprüft werden,dass diese mit Freizeitangebot nicht korrelieren. Anderes gesagt Datum stellt keine Varianz und für das Id stellt eine sehr große Varianz, wo es für unser Model nicht gut macht.<br>
Es gibt eine andere Lösung, wo wir das Datum in inserem modell rein bringen Können. Das wird in den letzten Kapitel hinausgegangen werden
<br>
<br>


In [3]:
df.corr()

# Only compute pearson prod-moment correlations between feature
# columns and target column
# print(pearsonr(df["Temperatur"], df["Freizeitangebot"]))

# label_encoder = LabelEncoder()
# df['Freizeitangebot'] = label_encoder.fit_transform(df['Freizeitangebot'])
# df['Datum'] = label_encoder.fit_transform(df['Datum'])

# X = df.drop("Datum",axis=1)
X = df.drop(["Freizeitangebot","Datum"],axis=1)
y = df["Freizeitangebot"]

## F-score, P-Wert and Mutal Information
f_score, f_p_value = f_classif(X,y)
mut_info_score = mutual_info_classif(X,y)
print('F - score score   ', f_score)
print('F - score p-value ', f_p_value)
print('mutual info       ', mut_info_score)

,id,relative Luftfeuchtigkeit,Temperatur,Niederschlag,Schnee
id,1.000000,0.127613,-0.119057,0.061902,-0.083225
relative Luftfeuchtigkeit,0.127613,1.000000,0.002101,-0.002072,0.019004
Temperatur,-0.119057,0.002101,1.000000,0.044874,-0.002350
Niederschlag,0.061902,-0.002072,0.044874,1.000000,-0.017246
Schnee,-0.083225,0.019004,-0.002350,-0.017246,1.000000


F - score score    [ 0.21138507  0.74362453  0.83957659 17.94913989  9.1204934 ]
F - score p-value  [9.31572220e-01 5.64505665e-01 5.03260647e-01 3.88720877e-11
 2.53247238e-06]
mutual info        [0.44378418 0.20907454 0.8697371  0.27471239 0.21744577]


In [4]:
data=df
# Umstellung und Überprüfung des Datums 
data["Datum"]=pd.to_datetime(data["Datum"]).dt.date

print(np.max(data.Datum))
print(np.min(data.Datum))
data.head(105)
data["Datum"] = np.where(data["Datum"]==np.max(data.Datum), np.datetime64("2012-08-18"),data["Datum"])
print(np.max(data.Datum))
print(np.min(data.Datum))

2015-08-18
2011-01-01


,id,Datum,relative Luftfeuchtigkeit,Temperatur,Niederschlag,Schnee,Freizeitangebot
0,1,2011-01-01,90.50,-0.6,0.0,58.0,Skifahren
1,2,2011-08-01,90.79,6.0,0.0,13.0,Wandern
2,3,2011-01-15,80.29,7.6,0.1,0.0,Wandern
3,4,2011-01-22,84.79,-2.9,0.0,2.0,Skifahren
4,5,2011-01-29,87.13,-7.5,0.0,2.0,Skifahren
...,...,...,...,...,...,...,...
100,101,2012-01-12,90.67,-3.9,0.0,8.0,Skifahren
101,102,2012-08-12,86.46,-9.0,0.0,6.0,Skifahren
102,103,2012-12-15,89.08,2.5,2.1,22.0,Skifahren
103,104,2012-12-22,95.04,1.4,4.6,2.0,Lesen


2012-12-29
2011-01-01


In [0]:
## neuer Datensatz
data = data.drop(["id"],axis=1)
## Erstellung des Datums als Index 
data.set_index("Datum", inplace=True)
# Daten Ordenen nach Datum
data.sort_values(["Datum"], axis=0, 
                 ascending=True, inplace=True)

In [6]:
# Ausgabe der Fieldern unserer Daten
data.columns

Index(['relative Luftfeuchtigkeit', 'Temperatur', 'Niederschlag', 'Schnee',
       'Freizeitangebot'],
      dtype='object')

<br>
<br>Wir sehen, dass die Daten auch Datum enthalten. Mit Datum ist relativ simple Daten darzustellen als eine Serie und fehlende Variable zu finden.
<br>
<br>
<br>

In [0]:
## Daten summary 
data.describe()

,relative Luftfeuchtigkeit,Temperatur,Niederschlag,Schnee
count,105.000000,105.000000,105.000000,105.000000
mean,83.752000,0.400000,1.192381,1.524762
std,71.323336,98.906936,3.093377,6.470418
min,43.290000,-999.000000,0.000000,0.000000
25%,68.830000,3.200000,0.000000,0.000000
50%,78.000000,8.900000,0.000000,0.000000
75%,85.290000,17.000000,0.700000,0.000000
max,792.100000,27.900000,18.900000,58.000000


In [0]:
#Überprüfung der fehlenden Daten
data.isna().sum()

relative Luftfeuchtigkeit    0
Temperatur                   0
Niederschlag                 0
Schnee                       0
Freizeitangebot              0
dtype: int64

In [7]:
## Überprüfung einzelne Datum
print(df.shape)
## Ploten Datenfieldern
fig = go.Figure([go.Bar(x= data.Freizeitangebot.unique(), y=data.Freizeitangebot.value_counts())])
fig.show()

(105, 7)


In [8]:
## scatter Plot matrix 
fig = px.scatter_matrix(data,
    dimensions=[ 'relative Luftfeuchtigkeit', 'Temperatur',
       'Niederschlag', 'Schnee'],
    color="Freizeitangebot")
fig.show()


-   Aus diesem Summary wird einige Informationen über Daten gegeben. wir haben 105 Datensätze und keine fehelende Daten. 
-   Die Aktivitäten sind 5 ('Skifahren', 'Wandern', 'Lesen', 'Fussball', 'Baden') und das am meistens wird Ski gefahren und wandern gegangen. <br>Das gibt uns eine Information, dass Kurt in einem Ort, wo es kalt ist.
-   Aus dem Matrix Plot sehen wir das die variable sehr gering miteinander korrelieren.
-   was noch zu bemerken ist, dass der Max von Luftfreutigkeít, Min von Temperature und auch Max von Schnee sehr stark abweichen. wodurch es Ausreißen gibt oder falsche Werte angemessen sein sollen. Prüfen wir Das durch ein paar Plots. 
<br>
<br>



In [9]:
#box plot beschtätigung für Aus reisse
fig1 = px.box(df, x="Freizeitangebot", y="Temperatur", points="all")
fig1.show()
fig2 = px.box(df, x="Freizeitangebot", y="Schnee", points="all")
fig2.show()
fig3 = px.box(df, x="Freizeitangebot", y="relative Luftfeuchtigkeit", points="all")
fig3.show()
fig4 = px.box(df, x="Freizeitangebot", y="Niederschlag", points="all")
fig4.show()

Der nächste Schritt ist die Daten zu visualisieren und ein paar inforamtionen über unsere Daten und Datenqualität zu bekommen

### Daten Visualisierung:


- Nach der Indizierung der Daten ist es jetzt möglich die Variablen in einer Zeitreihe darzustellen 

In [10]:
## visiualse the Tempertur in the Time
#data["Datum"]=pd.to_datetime(data["Datum"].astype(str), format='%m/%d/%Y')
fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['Temperatur'],
                name="Temperatur",
                line_color='deepskyblue',
                opacity=0.8)])
fig.show()

In [11]:
## visiualse the Schnee in the Time
fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['Schnee'],
                name="Schnee",
                line_color='#ff7f0e',
                opacity=0.8)])
fig.show()

In [12]:
## visiualse the relative Luftfeuchtigkeit in the Time
fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['relative Luftfeuchtigkeit'],
                name="relative Luftfeuchtigkeit",
                line_color='#2ca02c',
                opacity=0.8)])
fig.show()

In [13]:
## visiualse the Niederschlag in the Time
fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['Niederschlag'],
                name="Niederschlag",
                line_color='#8c564b',
                opacity=0.8)])
fig.show()

-   Wir sehen deutlich, dass die Temperatur am 22.09.2012 mit -999 grad falch angemessen und das stört unsere Datenqualität.
-   Für Schnee sehen wir das am 01.01.2011 sehr hoch ist und am 02.07.2011 sehr unwarscheilich das es schneit in dieser Zeit (Sommer)
-   Falsche Messung der relativen Luftfreutigkeit am 13.10.2012, weil der Wert sehr hoch als 100 und auch 02.04.2011 das gleiche.
-   Es fält schwer zu sagen ob es die Einheit nicht gegeben ist !!!! 

## Data Preparation
Nach der Prüfumg der Datenqualität, werden die Daten nun vorbereitet, um das Modell KNN durchzuführen.

Für vorher gennante Probleme werden die Falsche Messungen mit dem Median ersetzt(Man kann andre mögliche Lösungen nehmen)

In [0]:
# Falsche Daten löschen
#q = data["Temperatur"].quantile(0.99)
#data = data[data["Temperatur"] < q]

In [14]:
## replace the outlieres in the Temeratur
median_Temperatur = data.loc[data["Temperatur"] > -10 , 'Temperatur'].median()
data["Temperatur"] = np.where(data["Temperatur"] <-8 , median_Temperatur,data['Temperatur'])
fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['Temperatur'],
                name="Temperatur",
                line_color='deepskyblue',
                opacity=0.8)])
fig.show()

In [15]:
#replace the outliers of the snow

median_Schnee = data.loc[data["Schnee"] < 15 , 'Schnee'].median()

data["Schnee"] = np.where(data["Schnee"] > 15 , median_Schnee,data['Schnee'])

fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['Schnee'],
                name="Schnee",
                line_color='#ff7f0e',
                opacity=0.8)])
fig.show()

In [16]:
#replace the outliers of relativ umedity 

median_Luftfeuchtigkeit = data.loc[data["relative Luftfeuchtigkeit"] < 100 , 'relative Luftfeuchtigkeit'].median()

data["relative Luftfeuchtigkeit"] = np.where(data["relative Luftfeuchtigkeit"] > 100 , median_Luftfeuchtigkeit,data['relative Luftfeuchtigkeit'])

fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['relative Luftfeuchtigkeit'],
                name="relative Luftfeuchtigkeit",
                line_color='#CD5C5C',
                opacity=0.8)])
fig.show()


In [17]:
#replace the outliers of the Niedelschlage

median_Luftfeuchtigkeit = data.loc[data["Niederschlag"] < 7 , 'Niederschlag'].median()

data["Niederschlag"] = np.where(data["Niederschlag"] > 7 , median_Luftfeuchtigkeit,data['Niederschlag'])

fig = go.Figure([go.Scatter(
                x=data.index,
                y=data['Niederschlag'],
                name="Niederschlag",
                line_color='#800080',
                opacity=0.8)])
fig.show()

In [18]:
## box plot for the Variabls
fig1 = px.box(data, x="Freizeitangebot", y="Temperatur", points="all")
fig1.show()
fig2 = px.box(data, x="Freizeitangebot", y="Schnee", points="all")
fig2.show()
fig3 = px.box(data, x="Freizeitangebot", y="relative Luftfeuchtigkeit", points="all")
fig3.show()
fig4 = px.box(data, x="Freizeitangebot", y="Niederschlag", points="all")
fig4.show()

In [0]:
data.to_csv('neu_Daten.csv',index=False)

## Modeling
Jetzt, nachdem wir unsere Daten gereinigt haben, können wir Das KNN Model aufbauen und unsere Daten Trainieren und Testen.

Für Das KNN Modell gibt es einige Parametern, die wir einstellen müssen und optimieren sollen, damit Das Model besser die Vorhersagen besser trifft. <br>
Diese Parametern sind :
-  Anzahl der nächsten Nachbarn
-  Distanzmaß(Euklid,Hamming,..)
<br>
nicht zu vergessen, dass unsere metrische Variablen unterschiedlich skaliert sind, deshalb normalisieren wir erst mal unsere Daten, befor wir die Daten zerliegen.
 

Für das Parameter K nehmen wir zufälig K = 5
und für Distanzmaße nehemen wir den eukidischen.

In [36]:
#Erstellung labelEncoder
# le = LabelEncoder()
# Konvertierung string labels zu Zahl.
# Freizeitangebot_encoded=le.fit_transform(data.Freizeitangebot)
Freizeitangebot_encoded = data.Freizeitangebot
data_cd = data.drop("Freizeitangebot",axis=1)
# print(data_cd)

## Teilung der Daten in Test und Training
X_train, X_test, y_train, y_test = train_test_split(data_cd, Freizeitangebot_encoded , test_size=0.3) # 70% training and 30% test

### Normalisieren der Daten
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [37]:
#KNN Classifier mit normaler Gewichtung und euklidischem Distanzmaß default(P = 2)
knn = KNeighborsClassifier(n_neighbors=5,weights="uniform")

#Trainieren des Models
knn.fit(X_train, y_train)

#Vorhersagen mit Testdaten
y_pred = knn.predict(X_test)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

## Evaluation 


In [38]:
## Evaluation 
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3 0 0 0 0]
 [0 8 1 0 1]
 [0 0 2 1 2]
 [0 0 1 3 1]
 [0 0 0 0 9]]
              precision    recall  f1-score   support

       Baden       1.00      1.00      1.00         3
    Fussball       1.00      0.80      0.89        10
       Lesen       0.50      0.40      0.44         5
   Skifahren       0.75      0.60      0.67         5
     Wandern       0.69      1.00      0.82         9

    accuracy                           0.78        32
   macro avg       0.79      0.76      0.76        32
weighted avg       0.80      0.78      0.78        32



In [39]:
# Messung der Error für K  zwischen  1 and 10
Error=[]
for i in range(1, 10,2):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    Error.append(np.mean(pred_i != y_test))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [40]:
## Plot der Error mit Anstieg der Anzahl von K Werten

fig = go.Figure(go.Scatter(x=np.arange(1, 10,2), y=Error,mode='lines+markers'))

# Edit the layout
fig.update_layout(title='Error mit Anstieg der Anzahl von K Werten',
                   xaxis_title='K Wert',
                   yaxis_title='Mean Error')


Wie es  auf dem Diagram zu sehen ist, Dass  der Klassifikationsfehler mit K werte von 1 bis 5 runtergeffalen und ab 5 hochgegangen ist bzw  das Model effizienter mit Abstieg des K und ungenauer mit dem Aufstieg des K wird.<br>
Der Parameter K hat einen Einfluss auf unser Model, womit wir die Anzahl der nächste Nachbarn betrachten. Sobald die nächsten Nachbarn bekannt sind, werden die Anteile der auftretenden Labels berechnet. Das am häufigsten aufgetretene Label bestimmt die Klasse des Anfragepunktes 
<br> Für das Model ist es zu Empfehlen ein K zwischen 5 zu betrachten.



Um unser Model Robuster zu machen, sollen wir die Parametern verbessern.

Eine Lösung ist Krossvalidierung nehmen wir 10-Fache. Und zweite nehmen wir gewichtet Distanz. <br> 
wir sehen !, das Model performanter wird. wir nehmen K = 5.

In [24]:
# Krossvalidierung 
X = np.concatenate((X_train,X_test))

y = np.concatenate((y_train,y_test))
#Erstellung neues KNN Model
knn_cv = KNeighborsClassifier(n_neighbors=5)
#Model trainieren mit CV = 10
cv_erg = cross_val_score(knn_cv, X, y, cv=10,scoring='accuracy')
#CV Ergebnis (accuracy) und Durchschnitt sie
print(cv_erg)
print("cv_erg mean:{}".format(np.mean(cv_erg)))

[0.58333333 0.41666667 0.58333333 0.91666667 0.72727273 0.7
 0.8        0.8        0.75       0.5       ]
cv_erg mean:0.6777272727272727


In [28]:
## Vergleich 

k_range = range(1, 10,2)

# Leere Liste für Ergebnis
k_erg = []

# 1. we will loop through reasonable values of k
for k in k_range:
    # 2. run KNeighborsClassifier with k neighbours
    knn = KNeighborsClassifier(n_neighbors=k)
    # 3. obtain cross_val_score for KNeighborsClassifier with k neighbours
    erg = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    # 4. append mean of scores for k neighbors to k_scores list
    k_erg.append(erg.mean())

print(k_erg)


[0.6968939393939394, 0.7293939393939395, 0.6777272727272727, 0.7034848484848485, 0.7252272727272727]


In [29]:
print('Length of list', len(k_erg))
print('Max of list', max(k_erg))

Length of list 5
Max of list 0.7293939393939395


In [30]:
## Plot der Error mit Anstieg der Anzahl von K Werten

fig = go.Figure(go.Scatter(x=np.arange(1, 10,2), y=k_erg,mode='lines+markers'))

# Edit the layout
fig.update_layout(title='accuracy mit Anstieg der Anzahl von K Werten',
                   xaxis_title='K Wert',
                   yaxis_title='accuracy')

Nach alle diese Ausfürungen sehen wir das Krossvalidierung method deutlich die Accurancy des Model verringert im Vergleich zu normalem Training.<br>
und mit dieser Methode erhalten wir ein beste K = 3 von mit Accurancy von 73% <br>
Es liegt daran, dass Krossvalidierung den Überfetting vergingert deshalb bekommen wir ein geringeres Accurancy.

Dieses Mal versuchen wir mit dem gewichtem Abstand zu spielen und schauen wir die Ergebnisse.

In [47]:
## Gewichteter Abstand und Krossvalidierung

# KNN Classifier mit normaler Gewichtung und euklidischem Distanzmaß default(P = 2)
knn2 = KNeighborsClassifier(n_neighbors=3,weights="distance")

#Trainieren des Models 
knn2.fit(X_train, y_train)

#Predict
y_pred2 = knn2.predict(X_test)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [48]:
## Evaluation 
print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))

[[3 0 0 0 0]
 [1 7 1 0 1]
 [0 0 2 0 3]
 [0 0 1 2 2]
 [0 0 0 1 8]]
              precision    recall  f1-score   support

       Baden       0.75      1.00      0.86         3
    Fussball       1.00      0.70      0.82        10
       Lesen       0.50      0.40      0.44         5
   Skifahren       0.67      0.40      0.50         5
     Wandern       0.57      0.89      0.70         9

    accuracy                           0.69        32
   macro avg       0.70      0.68      0.66        32
weighted avg       0.73      0.69      0.68        32



In [54]:
# Messung der Error für K  zwischen  1 and 10
Error2=[]
for i in range(1, 10,2):
    knn2 = KNeighborsClassifier(n_neighbors=i)
    knn2.fit(X_train, y_train)
    pred_i2 = knn2.predict(X_test)
    Error2.append(np.mean(pred_i2 != y_test))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [55]:
## Plot der Error mit Anstieg der Anzahl von K Werten

fig = go.Figure(go.Scatter(x=np.arange(1, 10,2), y=Error2,mode='lines+markers'))

# Edit the layout
fig.update_layout(title='Error mit Anstieg der Anzahl von K Werten',
                   xaxis_title='K Wert',
                   yaxis_title='Mean Error')

In [56]:
# Krossvalidierung 
X2 = np.concatenate((X_train,X_test))

y2 = np.concatenate((y_train,y_test))
#Erstellung neues KNN Model
knn_cv2 = KNeighborsClassifier(n_neighbors=5)
#Model trainieren mit CV = 10
cv_erg2 = cross_val_score(knn_cv2, X2, y2, cv=10,scoring='accuracy')
#CV Ergebnis (accuracy) und Durchschnitt sie
print(cv_erg2)
print("cv_erg mean:{}".format(np.mean(cv_erg2)))



[0.75       0.58333333 0.58333333 0.66666667 0.72727273 0.7
 0.8        0.8        0.875      0.75      ]
cv_erg mean:0.7235606060606061


In [57]:
## Vergleich 

k_range = range(1, 10,2)

# Leere Liste für Ergebnis
k_erg2 = []

# 1. we will loop through reasonable values of k
for k in k_range:
    # 2. run KNeighborsClassifier with k neighbours
    knn2 = KNeighborsClassifier(n_neighbors=k)
    # 3. obtain cross_val_score for KNeighborsClassifier with k neighbours
    erg2 = cross_val_score(knn2, X2, y2, cv=10, scoring='accuracy')
    # 4. append mean of scores for k neighbors to k_scores list
    k_erg2.append(erg2.mean())

print(k_erg2)

[0.7003030303030303, 0.7402272727272727, 0.7235606060606061, 0.7277272727272728, 0.7318939393939393]


In [58]:
## Plot der Error mit Anstieg der Anzahl von K Werten

fig = go.Figure(go.Scatter(x=np.arange(1, 10,2), y=k_erg2,mode='lines+markers'))

# Edit the layout
fig.update_layout(title='accuracy mit Anstieg der Anzahl von K Werten',
                   xaxis_title='K Wert',
                   yaxis_title='accuracy')

Für die gleiche ausführung haben wir haben wir das gleiche ergebnis. Erstens war Training mit k=3 aber model sagt, dass K = 5 besser. Um das zu beschtätigen,
ist eine Krossvalidierung durchgefürt. und es stellt sich fest, dass wie am Anfang mit dem gleichen Datensatz (Training und Test) das beste K  ist gleich 3 mit einem Accurency 74% also mehr als das erste.
Das K Parameter hat den Einfluss auf die Güte, da jenach K wird der Anzahl der nächsten Nachbarn bestimmt und damit die Klassifikation auch so z.b für kleine K wird nur die Nachbern in der näher angesprochen und damit macht die Klassifikation nicht so robuster
Für größeren K ist wird das Model auch nicht Robuster, da wird auch die weitentfernten Nachbarn angenomen und die können Ausreißer sein, deshalb ist immer empholen ein K zu nehmen, das best für das Model passt, in unserem Fall ist K = 3

Jetzt Kommen wir zu dem Datum in unseren Daten. Es gibt Möglichkeiten das Datum mit in das Model eingeben und zwar :
-  wir können Das Jahr löchen und bleiben wir bei den Monaten oder Tagen, womit wir diese danach zu Monaten(Jan,Feb,...) oder Wochentagen(Mo,Di,..) transformieren oder per quartal, da schon die Temperatur mit der Zeit irgendwie eine Saisonnier hat und das Gleiche für andere Variablen.

-  Neben der Erste Lössung wir können so Buckets von Variablen z.b. (Datum zwischen X und Y wird Bücher gelesen usw)

## In to Production 

Nachdem wir die Daten gereinigt, das Model gebaut haben und den Parameter K bestimmt haben ist jetzt die Phase, wo wir das Model in die Produktion bringen.
<br>
die Produktion basiert auf kleien gehostet Web service mit Flask, womit wir das Model als Api stellen und einfach zu verwenden für die Vorhersage 

In [83]:
### das Model Packen 

## Prameter K = 3 und Distanz Gewichtet
# KNN Classifier mit normaler Gewichtung und euklidischem Distanzmaß default(P = 2)
knn3 = KNeighborsClassifier(n_neighbors=3,weights="distance")

#Train the model using the training sets
knn3.fit(X_train, y_train)

with open('knn_gewicht.pkl', 'wb') as handle:

    pickle.dump(knn3, handle, pickle.HIGHEST_PROTOCOL)

## Prameter K = 3 und Distanz normal
# KNN Classifier mit normaler Gewichtung und euklidischem Distanzmaß default(P = 2)
knn4 = KNeighborsClassifier(n_neighbors=3,weights="uniform")

#Train the model using the training sets
knn4.fit(X_train, y_train)

with open('knn_uniform.pkl', 'wb') as handle:

    pickle.dump(knn4, handle, pickle.HIGHEST_PROTOCOL)



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

Die beide Modele (gewicht und uniform) sind bereit in Flask als api zu ertellen. Es geht jetzt weiter mit der Datei app.py wo wir die Modele mit der Webseite verbinden


